## crime scraping

In [42]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import cpca
import jieba
from china_cities import *
import regex
import pinyin
import csv

r = requests.get("https://cnwoman-bot.github.io/evil-man/")
soup = BeautifulSoup(r.text,'html.parser')
soup.prettify()


## getting and modifying the crime list
def get_news_within(div):
    p = div.next_sibling.next_sibling
    lst_text = [a.text for a in p.find_all('a')]
    return lst_text

text = [i.parent.text.split('\n') for i in soup.find_all('a') if i.text.startswith('判决日期')]
o = []
for t in text:
    if len(t) <= 3:
        o.append(t[0])
    else:
        for s in t:
            if '判决日期' not in s:
                o.append(s)
o = list(set(o))
other1 = o

other2 = ['新疆喀什大学文学院援疆博士王明科','云南云南大学文学院副教授蔡英杰性骚扰留学生被公开处分，现任福建师范大学教授','甘肃兰州大学大气科学学院副院长张文煜被曝性侵',
         '上海戏剧学院教务处处长厉震林性骚扰，已无链接','央美设计学院院长宋协伟被指控性骚扰、猥亵女学生', '原北京中央民族大学文学院与新闻传播学院刘立刚性骚扰',
         '湖北美院偷拍女生裙底把视频发到了p站，无语，链接不放了','北京电影学院阿廖莎性侵案','北京电影学院侯亮平事件']

div = [d for d in soup.find_all('div',class_="language-plaintext highlighter-rouge") if '本小节合计' in d.text]
crime = []
for d in div:
    crime += get_news_within(d)
crime += other1
crime += other2
l = ['亳州6·15杀人案,邻居眼中的老实人,为何连杀4人','四川一名大学本科男子刘某在成都开往深圳的火车上猥亵9岁女童','浙大学生努某某强奸被判一年半，学校不与开除，留校察看','猥亵女童的网红“豆浆王子”蒙顺宁，出狱后获得公益组织爱心帮扶',
 '西南某大学法学院22岁男生偷拍女生裙底',
 '陕西安康石泉30岁的男子索爱一女子不成跳江自杀，女子被判决赔偿各项损失七万元']
crime += l
crime = [c for c in crime if c != '知乎问题'] #this is the name of the website; the crime has been manually added
crime = [l for l in crime if '台湾' not in l] #removing these bc they are controversial?

#manually dropping unqualified entries
manual_drop = ['澳洲华人男子罗浩灵疑残忍杀妻，藏尸公寓冰柜！带俩娃逃回国，刚刚被捕',
 '澳洲华人男子胡伟杀妻后自杀，8岁儿子在旁目睹全过程',
 '美国纽约州威彻斯特郡46岁的华裔男子刘传凯杀死妻子和俩孩子后自杀身亡',
 '新加坡一中国籍男子Cui Huan谋杀妻子，罪成将面对死刑或终身监禁兼鞭刑',
 '泰国华裔男子喂刚出生7天的女儿洁厕剂，杀害并埋尸坟地',
 '留日女学生江歌被闺蜜刘鑫前男友陈世峰捅刀杀死',
 '澳大利亚国立大学中国留学生Junqi Huang浴室偷拍被捕，遭校方开除。曾任超级脑学会的前任主席',
 '美国伊利诺伊大学厄巴纳-香槟分校华人教授徐钢性侵女学生','我是包丽的朋友，真相远比你知道的更可怕',
 '日本东京街头一中国籍男子强摸女性被捕',
 '女友爆料罗志祥多次出轨，参与多人群p',
 '女医护人员剃光头“是否自愿”遭质疑，网友：唯一男医生是平头','只要我不报道女性，不宣传女性，所有的功劳就还是男性的',
 '“洋媳妇”与“洋女婿”双重标准纪实',
 '各项扶贫政策落实后，某贫困户要求给他分配个女人',
 '“致良知四合院”会场宣传女性应该完全服从男性',
 '女德班成果：“中国女孩只属于中国男孩”','泰国乌汶府Pha Taem国家公园一江苏男子俞某杀妻谋财',
 '泰国春武里府的海滩，中国台湾男子卢子扬杀妻抛尸',
 '泰国一家酒店的泳池中天津男子张轶凡杀妻骗保',
 '韩国一中国籍游客男子陈某在天主教堂杀害一名女路人']
#these are dropped bc they either dont contain crime info or happened outside china
crime = [c for c in crime if c not in manual_drop]
#dropping cases happened on transportation where city is diffult to identify
transport = [c for c in crime if '开往' in c or ('火车' in c and '火车站' not in c)]
crime = [c for c in crime if c not in transport]

In [198]:
len(crime)

1773

In [199]:
## getting the city dictionary
url = "https://raw.githubusercontent.com/adyliu/china_area/master/area_code_2020.json"
#df = pd.read_json(url)
dct_city={}
for row in df.iterrows():
    prov = row[1]['name']
    for dct in row[1]['children']: #only consider city level = 1 and 2
        if dct['name'] == '市辖区' and dct['level'] == 2:
            key = row[1]['name']
        elif dct['level'] == 2:
            key = dct['name']
        if key.endswith('行政区划') == False and len(key) > 1: # this is because after adding the measure word the name has to have at least two characters
            rel_city = [d['name'] for d in dct['children']]
            rel_city.append(prov)
            sim_city = [c[:-1] for c in rel_city if len(c) >= 3]# this is to complement situations where ppl dont say the final measure word for short
            rel_city += sim_city
            rel_city.append(key[:-1])
            dct_city[key] = set(rel_city)
dct_city['香港特别行政区'] = {'香港', '香港特别行政区'}# manually adding these two special cities
dct_city['澳门特别行政区'] = {'澳门', '澳门特别行政区'}



In [45]:
## getting uni dct

url = "https://raw.githubusercontent.com/WenryXu/ChinaUniversity/master/ChinaUniversityList.json"
#df_uni = pd.read_json(url)
dct_uni = {}
for l in df_uni.schools:
    for d in l:
        c = d['city']
        n = d['name']
        l_c = len(c[:-1])
        if c in dct_uni.keys():
            dct_uni[c] += [n]
        else:
            dct_uni[c] = [n]

In [200]:
## matching crimes
## matching cases located in universities


left_crime = crime[:]
lst_ma = []
dct_m = {}


def match_uni(string,u):
    ind = [i for i in range(len(string)) if string[i] == u[0]][0]
    key = string[0:ind + 2]
    if key in lst_val:
        city = [k for k in dct_uni.keys() if key in dct_uni[k]][0]
        if city in dct_m.keys():
            dct_m[city] += 1
        else:
            dct_m[city] = 1
        if string in left_crime:
            left_crime.remove(string)
            lst_ma.append((string,city))
    else:
        set_w = jieba.cut(string)
        set_uni = {w for w in set_w if u in w}
        if len(set_uni) > 0:
            uni = [w for w in set_uni if len(w) == max({len(t) for t in set_uni})][0]
            if uni in lst_val:
                city = [k for k in dct_uni.keys() if uni in dct_uni[k]][0]
                if city in dct_m.keys():
                    dct_m[city] += 1
                else:
                    dct_m[city] = 1
                if string in left_crime:
                    left_crime.remove(string)
                    lst_ma.append((string,city))

lst_val = []
for v in dct_uni.values():
    lst_val += v
for l in crime:
    if '首都' in l or '中央' in l or '北大' in l or '清华' in l:
        if '北京市' not in dct_m.keys():
            dct_m['北京市'] = 1
        else:
            dct_m['北京市'] += 1
        left_crime.remove(l)
        lst_ma.append((l,'北京市'))
    if '大学' in l:
        match_uni(l,'大学')
    elif '学院' in l:
        match_uni(l,'学院')

#manually adding cases where university names are abbreviated which cannot be dealt with as these abbreviations are based on different patterns
uni_c = '央美国画系教授丘挺多次与多位女学生发生关系，其妻欲跳楼轻生'
try:
    left_crime.remove(uni_c)
    dct_m['北京市'] += 1
    lst_ma.append((uni_c,'北京市'))
except:
    pass

uni_c = '央美国画系教授姚舜熙被学生联合实名举报x骚扰、以权谋私、诽谤等'
try:
    left_crime.remove(uni_c)
    dct_m['北京市'] += 1
    lst_ma.append((uni_c,'北京市'))
except:
    pass

uni_c = '北航教授陈小武被举报性骚扰 疑似曾致手下学生怀孕'
try:
    left_crime.remove(uni_c)
    dct_m['北京市'] += 1
    lst_ma.append((uni_c,'北京市'))
except:
    pass

uni_c = '女学生遭HYD博导凌昌全教授十五年骚扰、打击报复、逼迫就范'
try:
    left_crime.remove(uni_c)
    dct_m['上海市'] += 1
    lst_ma.append((uni_c,'上海市'))
except:
    pass

uni_c = '中国音乐学院2017级男博士国某某要求他人偷拍女生洗澡被开除，起诉教委要求撤销处分'
try:
    left_crime.remove(uni_c)
    dct_m['北京市'] += 1
    lst_ma.append((uni_c,'北京市'))
except:
    pass

uni_c = '哈工大男生在网上用污言秽语辱骂师大女生'
try:
    left_crime.remove(uni_c)
    dct_m['哈尔滨市'] += 1
    lst_ma.append((uni_c,'哈尔滨市'))
except:
    pass

uni_c = '厦大海洋与海岸带发展研究院在读硕士彭炜敏婚内出轨，并假冒母亲身份欺骗女生'
try:
    left_crime.remove(uni_c)
    dct_m['厦门市'] += 1
    lst_ma.append((uni_c,'厦门市'))
except:
    pass

uni_c = '浙大央美性搔扰惯犯崔青洲'# i dont know where this uni is; can delete
try:
    left_crime.remove(uni_c)
    dct_m['杭州市'] += 1
    lst_ma.append((uni_c,'杭州市'))
except:
    pass

uni_c = '央美设计学院院长宋协伟被指控性骚扰、猥亵女学生'
try:
    left_crime.remove(uni_c)
    dct_m['北京市'] += 1
    lst_ma.append((uni_c,'北京市'))
except:
    pass

uni_c = '浙大学生努某某强奸被判一年半，学校不与开除，留校察看'
try:
    left_crime.remove(uni_c)
    dct_m['杭州市'] += 1
    lst_ma.append((uni_c,'杭州市'))
except:
    pass



In [201]:
len(lst_ma)


90

In [218]:
def cpca_match(s):
    df = cpca.transform([s])
    row = df.iloc[0,]
    city = row['市']
    prov = row['省']
    if prov != None and prov.endswith('市'):
        city = prov
        return city
    else:
        if city != None:
            if city.endswith('行政区划') == False:
                return city
        else:
            lst_w = list(jieba.cut(s))
            del_ind = len(lst_w[0])
            if len(s) > del_ind:
                city = cpca_match(s[del_ind:])
            return city

def county_match(s):
    row = cpca.transform([s]).iloc[0,]
    prov = row['省']
    if prov != None and row['市'] == None and row['区'] == None:
        s = row['地址']
    else:
        prov = None
    set_w = set(jieba.cut(s))

    if prov != None:
        potential = [c for c in dct_city.keys() if prov in dct_city[c]]
    else:
        potential = dct_city.keys()
    max_sim = 0
    cities = []
    city = None
    for k in potential:
        sim = len(dct_city[k] & set_w)
        if sim > max_sim:
            max_sim = sim
            cities = [k]
        elif sim != 0 and sim == max_sim:
            cities.append(k)
    if len(set(cities)) == 1:
        city = cities[0]
    else:
        lst_city = [] # this is seprated from above because some cases have more than one location in the description or wrongly matched higher and lower level locations
        key = list(jieba.cut(s))[0]
        county_sim = 0
        for c in potential:
            v = dct_city[c]
            ct_sim = len([ct for ct in v if key in ct])
            if ct_sim > county_sim:
                county_sim = ct_sim
                lst_city = [c]
            elif ct_sim != 0 and ct_sim == county_sim:
                lst_city.append(c)
        if len(lst_city) == 1:
            city = lst_city[0]
    return city


for c in left_crime:
    city = cpca_match(c)
    if city == None:
        city = county_match(c)
    if city != None:
        if city in dct_m.keys():
            dct_m[city] += 1
        else:
            dct_m[city] = 1
        left_crime.remove(c)
        lst_ma.append((c,city))




In [219]:
#8 times to get all results
len(lst_ma)

1611

In [220]:
## getting cities by retrieving info from the webpage
def from_link(url,s):
    city = None
    row = cpca.transform([s]).iloc[0,]
    prov = row['省']
    try:
        r_temp = requests.get(url)
        s_temp = BeautifulSoup(r_temp.text,'html.parser')
        set_compare = set([])
        if prov != None:
            c_compare = [k for k in dct_city.keys() if prov in dct_city[k]]
            for c in c_compare:
                set_compare = set_compare | dct_city[c]
        else:
            for v in dct_city.values():
                set_compare = set_compare | v

        set_shared = {w for w in list(jieba.cut(s_temp.text))} & set_compare

        if len(set_shared) == 1:
            key = list(set_shared)[0]
            if key in dct_city.keys():
                city = key
            else:
                city = [k for k in dct_city.keys() if key in dct_city[k]][0]
        else:
            cities = []
            for w in set_shared:
                c = [k for k in dct_city.keys() if w in dct_city[k]][0]
                cities.append(c)
            if len(cities) == 1:
                city = cities[0]
    except:
        city = None
    finally:
        return city

links = [a for a in soup.find_all('a') if a.text in left_crime and '.gov.cn' not in a['href']] #gov webpages cannot be scraped
for l in links: # this chunk can work faster under Chinese internet
    s = l.text
    url = l['href']
    if url.startswith('https:') == False:
        re.sub('http:','https:',url)
    city = from_link(url,s)
    if city != None:
        if city in dct_m.keys():
            dct_m[city] += 1
        else:
            dct_m[city] = 1
        left_crime.remove(s)
        lst_ma.append((s,city))



In [69]:
left_crime
len(lst_ma)
lst_ma

[('辽宁建昌县农贸市场宏大东北大药房一男子王某军持匕首当场捅死前妻', '北京市'),
 ('福建华侨大学回应江歌案：陈世峰是本校毕业生在校曾殴打女友', '泉州市'),
 ('北京工业大学2001级工商管理系男学生郭某某杀害女友被判无期后减刑9次，出狱后拒戴口罩打死老人', '北京市'),
 ('北京一快递员杨召朋假借收取快递包裹，入户抢劫殴打女子王某，刺切要害致死亡。女子为清华大学教师之妻', '北京市'),
 ('一男子在首都机场骚扰欧阳娜娜', '北京市'),
 ('湖南长沙中南大学南校升华公寓16栋下一男子王某割喉女子致死', '长沙市'),
 ('福建厦门大学环境与生态学院在读研究生荆某被28岁男子砸伤头部后去世，男子系东北某大学毕业生', '厦门市'),
 ('辽宁工程技术大学大一男生徐某某杀害一女生', '阜新市'),
 ('吉林大学中心校区经C楼附近一男子砍人至一死一伤', '长春市'),
 ('吉林大学大四男生李某勒死两卖淫女并辱尸，称和尸体做爱刺激', '长春市'),
 ('福建厦门一女患者在复旦大学附属医院厦门分院更衣室换衣服被男子观看全程，医院：不能说是偷窥！', '上海市'),
 ('兰州理工大学男子尾随厕所偷拍，罚款500', '兰州市'),
 ('上海东华大学04级研究生章胜偷窥女生如厕', '上海市'),
 ('江苏南京大学暑期支教女学生洗澡被偷拍，做案者仅罚款500元', '南京市'),
 ('江西农业大学一男生厕所内偷拍，官方：留校察看', '南昌市'),
 ('香港大学教师张祺忠杀妻', '香港特别行政区'),
 ('香港中文大学教授“毒球”谋杀妻女案', '香港特别行政区'),
 ('北大数学中心博导冯仁杰劈腿数十人，插足本科生、出轨已婚女', '北京市'),
 ('北大教授谢灿性骚扰', '北京市'),
 ('北大副教授余万里性侵女学生', '北京市'),
 ('北大客座教授、《南风窗》公益年度人物袁天鹏被曝涉嫌性侵女学员', '北京市'),
 ('原北大教师沈阳被指“性侵女生致其自杀”', '北京市'),
 ('北京财贸职业学院男辅导员魏雪金多次骚扰女学生并求交往', '北京市'),
 ('北京联合大学老师“袭胸”女学生', '北京市'),
 ('北京华北电力大学可再生能源学院院长戴松元，被曝性侵女教师', '北京市'

In [226]:
## manual edit

c = '香港粉岭一家再造纸工厂男子王某雄杀女同事后,放入碎纸机碎尸'
try:
    left_crime.remove(c)
    dct_m['香港特别行政区'] += 1
    lst_ma.append((uni_c,'香港特别行政区'))
except:
    pass

ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '福建福州大学三名男教师陈新、苏思文及李登峰非礼香港机场女地勤 从后方抱住袭胸，三人在福州大学分别任教化学、政治及管理'][0]
lst_ma[ind] = ('福建福州大学三名男教师陈新、苏思文及李登峰非礼香港机场女地勤 从后方抱住袭胸，三人在福州大学分别任教化学、政治及管理','香港特别行政区')
dct_m['香港特别行政区'] += 1
dct_m['福州市'] -= 1


ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '北工大外国语学院男生吴江掐死女友续：凶手被判死缓'][0]
lst_ma[ind] = ('北工大外国语学院男生吴江掐死女友续：凶手被判死缓','北京市')
dct_m['北京市'] += 1
dct_m['苏州市'] -= 1

ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '辽宁凌源籍男子占某在江苏南通市市区钟楼广场地下通道处偷袭猥亵过路女子'][0]
lst_ma[ind] = ('辽宁凌源籍男子占某在江苏南通市市区钟楼广场地下通道处偷袭猥亵过路女子','南通市')
dct_m['南通市'] += 1
dct_m['朝阳市'] -= 1

ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '甘肃青海大通县多林镇吴仕庄村男子蔡某与妻子不和，杀妻子全家后自杀'][0]
lst_ma[ind] = ('甘肃青海大通县多林镇吴仕庄村男子蔡某与妻子不和，杀妻子全家后自杀','西宁市')
dct_m['西宁市'] = 1
dct_m['重庆市'] -= 1


ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '辽宁建昌县农贸市场宏大东北大药房一男子王某军持匕首当场捅死前妻'][0]
lst_ma[ind] = ('辽宁建昌县农贸市场宏大东北大药房一男子王某军持匕首当场捅死前妻','葫芦岛市')
dct_m['葫芦岛市'] += 1
dct_m['北京市'] -= 1


ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '原北大教师沈阳被指“性侵女生致其自杀”'][0]
lst_ma[ind] = ('原北大教师沈阳被指“性侵女生致其自杀','南京市')
dct_m['南京市'] += 1
dct_m['北京市'] -= 1

ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '贵州紫云县一父亲强奸20岁亲生女儿长达6年 一审被判12年'][0]
lst_ma[ind] = ('贵州紫云县一父亲强奸20岁亲生女儿长达6年 一审被判12年','安顺市')
dct_m['安顺市'] = 1
dct_m['临沧市'] -= 1

ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '山东陵县宋家镇东屯村农民郭永庆掐死女友'][0]
lst_ma[ind] = ('山东陵县宋家镇东屯村农民郭永庆掐死女友','德州市')
dct_m['德州市'] += 1
dct_m['重庆市'] -= 1

ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '云南巍山县大仓镇男子段金泉杀害6名年轻女性'][0]
lst_ma[ind] = ('云南巍山县大仓镇男子段金泉杀害6名年轻女性','大理白族自治州')
dct_m['大理白族自治州'] += 1
dct_m['重庆市'] -= 1

l = '江西玉山一湖南籍男子张某猥亵女学生和女老师'
dct_m['上饶市'] += 1
left_crime.remove(l)
lst_ma.append((l,'上饶市'))

ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '俞渝爆李国庆有梅毒，行贿，转移财产送多个情人'][0]
del lst_ma[ind]
dct_m['乌海市'] -= 1

ind = [i for i in range(len(lst_ma)) if lst_ma[i][0] == '海南省东方市东河镇亚要村男子符李民持棍棒和石头追打母亲致死'][0]
del lst_ma[ind]
dct_m['乌海市'] -= 1

IndexError: list index out of range

In [235]:
len(left_crime) / len(crime)

0.0772701635645798

In [236]:
## matching to English names and writing into csv
dct_lang = {}

for c in cities.get_cities():
    if c.name_cn in dct_m.keys():
        dct_lang[c.name_cn] = (c.name_en,c.province)

lst_special = [c for c in dct_m.keys() if c not in dct_lang.keys()]


In [252]:
# this chunk of code needs to be run multiple times to finish editing, dont know why
for ct in lst_special:
    if ct.startswith('香港'):
        dct_lang[ct] = ('HongKong', 'Hongkong')
    elif ct.startswith('澳门'):
        dct_lang[ct] = ('Macao', 'Macao')
    else:
        key = list(jieba.cut(ct,cut_all=True))[0]
        eng = [(c.name_en,c.province) for c in cities.get_cities() if key in c.name_cn]
        if len(eng) == 1:
            dct_lang[ct] = eng[0]
            lst_special.remove(ct)
        else:
            eng_name = pinyin.get(key, format="strip", delimiter="").capitalize()
            prov = [c for c in dct_city[ct] if c.endswith('省')][0]
            prov_eng = pinyin.get(prov[:-1], format="strip", delimiter="").capitalize()
            dct_lang[ct] = (eng_name,prov_eng)
            lst_special.remove(ct)

In [254]:
lst_result = []
for k,v in dct_m.items():
    tup = (dct_m[k],k,dct_lang[k])
    lst_result.append(tup)
lst_result.sort()

with open("data.csv","w") as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(['number of cases','location_CN','location_ENG'])
    for l in lst_result:
        writer.writerow(l)

with open("matches.csv","w") as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(['case','location'])
    for l in lst_ma:
        writer.writerow(l)